In [1]:
#load and merge
import pandas as pd
import glob, os

raw_path = "../data_raw/*.csv"
files = sorted(glob.glob(raw_path))

dfs = []
for file in files:
    df_temp = pd.read_csv(file, low_memory=False)
    df_temp.columns = [c.strip().lower() for c in df_temp.columns]
    rename_map = {
        'started_at': 'started_at', 'start_time': 'started_at',
        'ended_at': 'ended_at', 'end_time': 'ended_at',
        'member_casual': 'member_casual', 'usertype': 'member_casual',
        'ride_id': 'ride_id', 'trip_id': 'ride_id',
        'rideable_type': 'rideable_type', 'bike_type': 'rideable_type'
    }
    df_temp = df_temp.rename(columns={k: v for k, v in rename_map.items() if k in df_temp.columns})
    dfs.append(df_temp)

df = pd.concat(dfs, ignore_index=True, sort=False)
print("Initial shape:", df.shape)
df.head()


Initial shape: (5860568, 13)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,C1D650626C8C899A,electric_bike,2024-01-12 15:30:27,2024-01-12 15:37:59,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.903267,-87.634737,41.889177,-87.638506,member
1,EECD38BDB25BFCB0,electric_bike,2024-01-08 15:45:46,2024-01-08 15:52:59,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.902937,-87.634440,41.889177,-87.638506,member
2,F4A9CE78061F17F7,electric_bike,2024-01-27 12:27:19,2024-01-27 12:35:19,Wells St & Elm St,KA1504000135,Kingsbury St & Kinzie St,KA1503000043,41.902951,-87.634470,41.889177,-87.638506,member
3,0A0D9E15EE50B171,classic_bike,2024-01-29 16:26:17,2024-01-29 16:56:06,Wells St & Randolph St,TA1305000030,Larrabee St & Webster Ave,13193,41.884295,-87.633963,41.921822,-87.644140,member
4,33FFC9805E3EFF9A,classic_bike,2024-01-31 05:43:23,2024-01-31 06:09:35,Lincoln Ave & Waveland Ave,13253,Kingsbury St & Kinzie St,KA1503000043,41.948797,-87.675278,41.889177,-87.638506,member


In [2]:
# Data cleaning and feature engineering
for col in ['started_at', 'ended_at']:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

df = df.dropna(subset=['started_at','ended_at'])

df['ride_length_s'] = (df['ended_at'] - df['started_at']).dt.total_seconds()
df = df[(df['ride_length_s'] > 0) & (df['ride_length_s'] <= 24*3600)]
df['ride_length_min'] = df['ride_length_s'] / 60

df['day_of_week'] = df['started_at'].dt.day_name()
df['hour'] = df['started_at'].dt.hour
df['month'] = df['started_at'].dt.month_name()

df['member_casual'] = df['member_casual'].str.lower().str.strip()
df['member_casual'] = df['member_casual'].replace({
    'subscriber':'member',
    'customer':'casual',
    'customers':'casual'
})

print("Cleaned shape:", df.shape)
df[['member_casual','ride_length_min','day_of_week']].head()


Cleaned shape: (1691108, 18)


,member_casual,ride_length_min,day_of_week
0,member,7.533333,Friday
1,member,7.216667,Monday
2,member,8.000000,Saturday
3,member,29.816667,Monday
4,member,26.200000,Wednesday


In [4]:
os.makedirs("E:/GitHub/CysclisticGrowthAnalysis/data_working", exist_ok=True)
df.to_csv("E:/GitHub/CysclisticGrowthAnalysis/data_working/cyclistic_cleaned.csv", index=False)
